In [1]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [2]:
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import math

In [22]:
link = 'drive/MyDrive/Colab Notebooks/Wild/Projet 2/CSV/EncCRYN_R.csv'
df = pd.read_csv(link, index_col=0)
link2 = 'drive/MyDrive/Colab Notebooks/Wild/Projet 2/CSV/KMeans_G_15.csv'
df2 = pd.read_csv(link2, index_col=0, usecols=[0, 1 ,4])
link3 = 'drive/MyDrive/Colab Notebooks/Wild/Projet 2/CSV/final_FR.csv'
df3 = pd.read_csv(link3, index_col=0, usecols=[0, 1 ,5])
df3.drop_duplicates(subset=['tconst'], inplace=True)

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54188 entries, 0 to 54187
Data columns (total 8 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   tconst         54188 non-null  object 
 1   titleFR        54188 non-null  object 
 2   EncRunT        54188 non-null  int64  
 3   EncYear        54188 non-null  int64  
 4   EncNumR        54188 non-null  int64  
 5   averageRating  54188 non-null  float64
 6   Crew1          54188 non-null  float64
 7   Crew2          54188 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 3.7+ MB


In [15]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 54188 entries, 0 to 54187
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tconst      54188 non-null  object
 1   titleFR     54188 non-null  object
 2   Cluster_15  54188 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 1.7+ MB


In [13]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66854 entries, 9 to 2490836
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   tconst     66854 non-null  object 
 1   startYear  66854 non-null  float64
dtypes: float64(1), object(1)
memory usage: 1.5+ MB


In [24]:
df4 = df.merge(df2, on=['tconst'])

In [25]:
df5 = df4.merge(df3, on=['tconst'])

In [28]:
df5.rename(columns={'titleFR': 'Titre', 'startYear': 'Année de sortie'}, inplace=True)
df5['Année de sortie']=df5['Année de sortie'].astype(int)

In [29]:
df5.head()

,tconst,Titre,EncRunT,EncYear,EncNumR,averageRating,Crew1,Crew2,Cluster_15,Année de sortie
0,tt0002423,Passion,1,0,0,6.8,0.000000e+00,0.000000e+00,11,1919
1,tt0008843,La Demi-lune rouge,1,0,0,5.9,0.000000e+00,0.000000e+00,1,1919
2,tt0008933,Les frères Karamazoff,0,0,0,6.5,0.000000e+00,0.000000e+00,1,1920
3,tt0009559,Le roman de la vallée heureuse,0,0,0,6.1,-4.150833e-19,2.708531e-19,8,1919
4,tt0009568,Rose-France,0,0,0,6.9,0.000000e+00,0.000000e+00,1,1919


In [45]:
def recoG15():
  list_tconst=[]
  film_user = input("Veuillez entrer un film : ")
  print("\n")
  besoin = df5['tconst'].loc[df5['Titre'].str.contains(film_user, case=False)].items()
  for code, tconst in besoin:
    list_tconst.append(tconst)
  
  if len(list_tconst) == 1:
    tconst_user = df5['tconst'].loc[df5['Titre'].str.contains(film_user, case=False)].item()
    cluster_user = df5['Cluster_15'].loc[df5['Titre'].str.contains(film_user, case=False)].item()
    


    X = df5[['EncRunT',	'EncYear', 'averageRating','EncNumR',	'Crew1',	'Crew2']].loc[df5['Cluster_15']==cluster_user]
    y = df5['tconst'].loc[df5['Cluster_15']==cluster_user]
    neigh=math.sqrt(len(X))
    if round(neigh, 0)%2 == 0:
      neigh = int(round(neigh+1, 0))
    else:
      neigh = int(round(neigh, 0))
    modelknn=KNeighborsClassifier(n_neighbors=neigh, weights='distance').fit(X, y)
    reco=modelknn.kneighbors(X.loc[df5['tconst']==tconst_user], n_neighbors=11)


    print("\n")
    print("Si vous avez aimé", df5['Titre'].loc[df5['tconst']==tconst_user].item(), "vous aimerez sûrement :\n")
    for nbfilms in range(11):
      idfilm = reco[1].reshape(len(reco[1][0]), 1)[nbfilms].item()
      reco_titre = df5['Titre'].loc[df5['Cluster_15']==cluster_user].iloc[idfilm]
      if reco_titre != df5['Titre'].loc[df5['tconst']==tconst_user].item():
        print(reco_titre)
      else:
        continue
    print("\n")
    print("Abonnez-vous !")

  elif len(list_tconst) > 1:
      print("Nous avons trouvé plusieurs films correspondant, merci d'indiquer le code (tconst) du film que vous désirez ci dessous.\n")
      print(df5[['tconst', 'Titre', 'Année de sortie']].loc[df5['tconst'].isin(list_tconst)].to_string(index=False))
      print("\n")
      tconst_user = input("Votre code (tconst) ici : ")

      try: 
        cluster_user = df5['Cluster_15'].loc[df5['tconst'] == tconst_user].item()
        

        X = df5[['EncRunT',	'EncYear', 'averageRating','EncNumR',	'Crew1',	'Crew2']].loc[df5['Cluster_15']==cluster_user]
        y = df5['tconst'].loc[df5['Cluster_15']==cluster_user]

        neigh=math.sqrt(len(X))
        if round(neigh, 0)%2 == 0:
          neigh = int(round(neigh+1, 0))
        else:
          neigh = int(round(neigh, 0))

        modelknn=KNeighborsClassifier(n_neighbors=neigh, weights='distance').fit(X, y)

        reco=modelknn.kneighbors(X.loc[df5['tconst']==tconst_user], n_neighbors=10)
        print("\n")
        print("Si vous avez aimé", df5['Titre'].loc[df5['tconst']==tconst_user].item(), "vous aimerez sûrement :\n")
        for nbfilms in range(10):
          idfilm = reco[1].reshape(len(reco[1][0]), 1)[nbfilms].item()
          reco_titre = df5['Titre'].loc[df5['Cluster_15']==cluster_user].iloc[idfilm]
          if reco_titre != df5['Titre'].loc[df5['tconst']==tconst_user].item():
            print(reco_titre)
          else:
            continue
      except:
        print('Code incorrect, veuillez recommencer.')

  else:
    print("Désolé, nous n'avons pas trouvé de film correspondant à votre recherche.")

In [46]:
recoG15()

Veuillez entrer un film : Le gendarme de Saint-Tropez




Si vous avez aimé Le gendarme de Saint-Tropez vous aimerez sûrement :

Jo
Female Trouble
Pair et impair
L'Abominable Dr. Phibes
Un hold-up extraordinaire
Les Russes arrivent, les Russes arrivent
Le cher disparu
Trafic
Ennemis comme avant
L'emmerdeur


Abonnez-vous !
